In [5]:
import sys, os
import pandas as pd
import dask
import dask.dataframe as dd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold ,RandomizedSearchCV ,train_test_split, cross_val_predict , cross_val_score
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,classification_report, confusion_matrix,accuracy_score,matthews_corrcoef

In [ ]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(seed = 101)

In [ ]:
# !ls

In [1]:
import xgboost as xgb

In [ ]:
xgb.XGBRegressor()

In [ ]:
big_df = pd.DataFrame()
all_files = [x for x in os.listdir(".") if ".csv" in x and x != "subchallenge_1_template_data.csv"  ]

In [ ]:
# all_files

In [ ]:
# data_to_fill = pd.read_csv("subchallenge_1_template_data.csv")

In [ ]:
# print (data_to_fill.columns)

In [ ]:
target_cell_lines = ['AU565', 'EFM19', 'HCC2218', 'LY2', 'MACLS2', 'MDAMB436']

In [ ]:
target_genes = ['p.ERK', 'p.Akt.Ser473.', 'p.S6', 'p.HER2', 'p.PLCg2']

In [ ]:
for m in all_files:
#     print (m)
    df_temp = dd.read_csv(m)
    big_df = dd.concat([big_df,df_temp],axis=0)

In [ ]:
train_df = big_df[~big_df["cell_line"].isin(target_cell_lines)]

In [ ]:
X = train_df.drop(target_genes,axis=1)

In [ ]:
y = train_df[target_genes]

In [ ]:

X = X.categorize(columns=["treatment"])

In [ ]:
# X["treatment"]
my_dummies = dd.get_dummies(X["treatment"])

In [ ]:
# X.columns

In [ ]:
X= X.drop(['treatment', 'cell_line', 'time', 'cellID', 'fileID'],axis=1)

In [ ]:
# y.columns

In [ ]:
X=X.compute()

In [ ]:
y=y.compute()

In [ ]:
for m in X.columns :
    X[m].fillna((X[m].mean()),inplace=True)

In [ ]:
for m in y.columns :
    y[m].fillna((y[m].mean()),inplace=True)

In [ ]:
# my_dict
scaler = StandardScaler()


In [ ]:
scaler.fit(X)
X_scaled = scaler.transform(X)


In [1]:
my_target_ditionary = {'ERK':'p.ERK', 'AKT':'p.Akt.Ser473.', 'S6':'p.S6', 'HER':'p.HER2', 'PLCG2':'p.PLCg2'}

In [ ]:
base_model = RandomForestRegressor( n_estimators = 1000, random_state = 101,n_jobs=-1)

In [4]:
my_results = []
for key in my_target_ditionary.keys():
    print (key,my_target_ditionary[key])
    X_train, X_test, y_train, y_test = train_test_split(X_scaled , y[key], test_size=0.33, random_state=101)
    base_model.fit(X_train, y_train)
    predictions = base_model.predict(X_test)
    print ("R^2:",r2_score(y_test, predictions))
    print ("MAE:",mean_absolute_error(y_test, predictions))
    print ("MSE:",mean_squared_error(y_test, predictions))
    r=r2_score(y_test, predictions)
    mae=mean_absolute_error(y_test, predictions)
    mse=mean_squared_error(y_test, predictions)
    print ("------------------------------------------")
    my_results.append((key,r,mae,mse))
    
    error = predictions - y_test
    plt.hist(error, bins = 25)
    plt.xlabel("Prediction Error [MAE]")
    _ = plt.ylabel("Count")
    plt.savefig("basemodel_error_bin_%s.png"%(key),format="png")
    

    plt.scatter(y_test, predictions)
    plt.xlabel('True Values [MAE]')
    plt.ylabel('Predictions [MAE]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])
    plt.savefig("MAE_base_model_%s.png"%(key),format="png")



ERK p.ERK
AKT p.Akt.Ser473.
S6 p.S6
HER p.HER2
PLCG2 p.PLCg2


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_scaled , y_ERK, test_size=0.33, random_state=101)

In [ ]:
print (pd.DataFrame(my_results))